In [1]:
from lib import *
import numpy as np

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
raw,key_list         = load_larsoft_root("RAW","data/opdetraw_hist.root",     "opdigiana",np.loadtxt("template/fbk_deco.txt"))
gauss_dec,key_list   = load_larsoft_root("DEC","data/deconv_gauss_hist.root", "opdecoana",np.loadtxt("template/fbk_deco.txt"))
wiener_dec,key_list  = load_larsoft_root("DEC","data/deconv_wiener_hist.root","opdecoana",np.loadtxt("template/fbk_deco.txt"))

In [3]:
SER = []
templates = ["fbk_digi.txt","fbk_deco.txt","fbk_1024_dapv2.txt","fbk_DAPHNE_Dec22.txt","fbk_CA_deco_sinpret.txt"]
for path in templates:
    SER.append(np.loadtxt("template/"+path))

fig = go.Figure()
for i in range(len(SER)):
    fig.add_trace(go.Scatter(name=templates[i],x=np.arange(len(SER[i])),y=SER[i]))

fig.update_layout(title_text='Template comparison',
                xaxis_title  = "Time in [ticks]",
                yaxis_title  = "Amp. in [ADC]",
)

fig.show()

*** VISUALIZE DATA ***

In [4]:
num = np.random.randint(len(raw["RECO"]["WVF"]))
# num = np.where(np.asarray(gauss_dec["RECO"]["CH"]) == 232)[0][0]
print(num)

color_map={"RAW": "#3366CC", "GAUSS": "#66AA00", "WIENER":"#FF9900"}
fig = make_subplots(rows=1, cols=2,subplot_titles=('', ''))

fig.add_trace(go.Scatter(line=dict(color=color_map["RAW"]),name="RAW #PE: %.2f"%(raw["RECO"]["PE"][num]),x=raw["RECO"]["WVF_X"][num],y=raw["RECO"]["WVF"][num]-raw["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map["GAUSS"]),name="GAUSS #PE: %.2f"%(gauss_dec["RECO"]["PE"][num]),x=gauss_dec["RECO"]["WVF_X"][num],y=gauss_dec["RECO"]["WVF"][num]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map["WIENER"]),name="WIENER #PE: %.2f"%(wiener_dec["RECO"]["PE"][num]),x=wiener_dec["RECO"]["WVF_X"][num],y=wiener_dec["RECO"]["WVF"][num]),col=2,row=1)

fig.add_vline(x=raw["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=1)
fig.add_vline(x=gauss_dec["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=2)

try:
    if len(raw["TRUE"]["PETIMES"][num]) > 50:
        fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(raw["TRUE"]["PETIMES"][num])),x=np.asarray(raw["TRUE"]["PETIMES"][num])+raw["PRETRIGGER"]*raw["SAMPLING"]*1e6),col=1,row=1)
    else:
        fig.add_trace(go.Scatter(mode="markers",line=dict(color="#DC3912"),name="TRUE #PE: %.2f"%(len(raw["TRUE"]["PETIMES"][num])),x=np.asarray(raw["TRUE"]["PETIMES"][num])+raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,y=np.zeros(len(raw["TRUE"]["PETIMES"][num]))),col=1,row=1)
        fig.add_trace(go.Scatter(mode="markers",line=dict(color="#DC3912"),name="TRUE #PE: %.2f"%(len(raw["TRUE"]["PETIMES"][num])),x=np.asarray(raw["TRUE"]["PETIMES"][num])+raw["PRETRIGGER"]*raw["SAMPLING"]*1e6,y=np.zeros(len(raw["TRUE"]["PETIMES"][num]))),col=2,row=1)
except:
    print("WVF WITHOUT TRUE PHOTONS")
    
fig.update_layout(
    title = "Deco. wvf comparison",
    xaxis_title  = "Time in [&mu;s]",
    yaxis_title  = "Amp. in [ADC]",
    xaxis2_title = "Time in [&mu;s]",
    yaxis2_title = "Amp. in [a.u.]",
)
fig.show()

112


*** CONFIGURE FILTER FOR ANALYSIS ***

In [5]:
pe_error_raw    = (np.asarray(raw["RECO"]["PE"])-np.asarray(raw["TRUE"]["PE"]))/np.asarray(raw["TRUE"]["PE"])
pe_error_gauss  = (np.asarray(gauss_dec["RECO"]["PE"])-np.asarray(raw["TRUE"]["PE"]))/np.asarray(raw["TRUE"]["PE"])
pe_error_wiener = (np.asarray(wiener_dec["RECO"]["PE"])-np.asarray(raw["TRUE"]["PE"]))/np.asarray(raw["TRUE"]["PE"])

t0_reco_raw     = np.asarray(raw["RECO"]["T0"])-raw["PRETRIGGER"]*raw["SAMPLING"]*1e6
t0_reco_gauss   = np.asarray(gauss_dec["RECO"]["T0"])-raw["PRETRIGGER"]*raw["SAMPLING"]*1e6
t0_reco_wiener  = np.asarray(wiener_dec["RECO"]["T0"])-raw["PRETRIGGER"]*raw["SAMPLING"]*1e6

amp          = np.append(raw["RECO"]["AMP"],np.append(gauss_dec["RECO"]["AMP"],wiener_dec["RECO"]["AMP"]))
pe_reco      = np.append(raw["RECO"]["PE"],np.append(gauss_dec["RECO"]["PE"],wiener_dec["RECO"]["PE"]))
pe_true      = np.concatenate([raw["TRUE"]["PE"],raw["TRUE"]["PE"],raw["TRUE"]["PE"]])
pe_error     = 100*np.concatenate([pe_error_raw,pe_error_gauss,pe_error_wiener])
t0_reco      = np.concatenate([t0_reco_raw,t0_reco_gauss,t0_reco_wiener])
filter_label = np.concatenate([["RAW"]*len(pe_error_raw),["GAUSS"]*len(pe_error_gauss),["WIENER"]*len(pe_error_gauss)])

df = pd.DataFrame({"FILTER":filter_label, "'%'ERROR PE":pe_error, "TRUE PE":pe_true, "RECO PE":pe_reco, "AMP [a.u.]":amp, "RECO T0":t0_reco})

/tmp/ipykernel_203746/2655589605.py:1: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_203746/2655589605.py:2: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_203746/2655589605.py:3: RuntimeWarning:

divide by zero encountered in divide



In [6]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 0)*(df['TRUE PE'] < 100)], y='TRUE PE', x="RECO PE",
color='FILTER',color_discrete_map=color_map, marginal_x='histogram',trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
# print(results)
results.query("FILTER == 'GAUSS'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     755.8
Date:                Sat, 11 Mar 2023   Prob (F-statistic):           7.04e-74
Time:                        20:05:49   Log-Likelihood:                -885.48
No. Observations:                 227   AIC:                             1775.
Df Residuals:                     225   BIC:                             1782.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.6057      1.415      6.787      0.000       6.817      12.395
x1             0.7823      0.028     27.492      0.000       0.726       0.838
==============================================================================
Omnibus:                        6.239   Durbin-Watson:                   1.751
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                6.879
Skew:                           0.260   Prob(JB):                       0.0321
Kurtosis:                       3.676   Cond. No.                         88.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 100)*(df['TRUE PE'] < 1000)], y="TRUE PE", x="'%'ERROR PE",
color='FILTER',color_discrete_map=color_map,marginal_x='box',template="plotly")
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_traces(opacity=0.75)
fig.show()

In [10]:
fig = px.histogram(data_frame=df[(df['TRUE PE'] > 0)*(df['TRUE PE'] < 1000)], x="RECO T0",
color='FILTER',color_discrete_map=color_map,barmode="group",marginal='box',template="plotly",nbins=3000,range_x=[-0.16,0.32])
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.show()